In [1]:
import xarray as xr
import numpy as np
import cftime
import nc_time_axis
import pandas as pd
import matplotlib.pyplot as plt 
from matplotlib import ticker, cm
from cftime import datetime 
import datetime as dt
import matplotlib.colors as colors
import math
import random
import matplotlib.cm as mcm
from matplotlib.ticker import MaxNLocator
#jet = mcm.get_cmap('jet')
jet = mcm.get_cmap('jet') if isinstance(mcm.get_cmap('jet'), str) else mcm.get_cmap('jet')
import netCDF4 as nc
import sys
import os
import psutil
import netCDF4 as nc
import cartopy.crs as ccrs # CRS stands for "Coordinate reference systems" for map projection
from cartopy.crs import PlateCarree
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from dateutil import tz
import pytz
import time
from time import process_time
from tqdm import tqdm
import dask.array as da
import dask.dataframe as dd
import dask
%matplotlib inline 
#import line_profiler
#%load_ext line_profiler

/tmp/ipykernel_498017/892673583.py:16: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  jet = mcm.get_cmap('jet') if isinstance(mcm.get_cmap('jet'), str) else mcm.get_cmap('jet')


In [2]:
filepath = 'Nc_Files/mod_s4max_80-130km.nc'

ds = xr.open_dataset(filepath)
ds

<xarray.Dataset>
Dimensions:   (nfile: 6449380)
Coordinates:
  * nfile     (nfile) int64 0 2 3 4 6 ... 9266527 9266531 9266534 9266535
    altitude  (nfile) float64 ...
    lat       (nfile) float64 ...
    lon       (nfile) float64 ...
    time      (nfile) datetime64[ns] ...
    season    (nfile) object ...
    month     (nfile) object ...
Data variables:
    Ut_time   (nfile) float64 ...
    DOY       (nfile) float64 ...
    LT        (nfile) float64 ...
    mlat      (nfile) float64 ...
    mlon      (nfile) float64 ...
    mlt       (nfile) float64 ...
    s4max     (nfile) float64 ...
    year      (nfile) float64 ...

In [3]:
# Extract variables and coordinates to new variables
altitude = ds['altitude']
lat = ds['lat']
lon = ds['lon']
time = ds['time']
season = ds['season']
month = ds['month']

Ut_time = ds['Ut_time']
DOY = ds['DOY']
LT = ds['LT']
mlat = ds['mlat']
mlon = ds['mlon']
mlt = ds['mlt']
s4max = ds['s4max']
year = ds['year']


In [7]:
#Creates a new xarray data array. 
#Specifically, mthavg_s4max is an xarray DataArray object containing the monthly mean values of the 's4max' variable
#Not sure if this is best way, as doesn't contain other coords/variables

# Group the data by the 'month' coordinate and calculate the mean
mthavg_s4max = ds['s4max'].groupby('month').mean(dim='nfile')

# 'mthavg_s4max' now contains the monthly mean values of 's4max'

In [14]:
grouped = ds.groupby(['lat', 'lon', 'time', 'season', 'month'])

TypeError: `group` must be an xarray.DataArray or the name of an xarray variable or dimension. Received ['lat', 'lon', 'time', 'season', 'month'] instead.

In [13]:
grouped

<xarray.DataArray 's4max' (altitude: 49941)>
array([0.43522222, 0.34350001, 0.30252941, ..., 0.3325    , 0.159     ,
       0.26849999])
Coordinates:
  * altitude  (altitude) float64 80.0 80.0 80.0 80.0 ... 130.0 130.0 130.0 130.0

In [8]:
# Extract latitude and longitude coordinates
latitude = ds_mean['lat']
longitude = ds_mean['lon']

# Create a contour or heatmap plot
plt.figure(figsize=(10, 6))
plt.contourf(longitude, latitude, ds_mean['s4max'], levels=20, cmap='viridis')
plt.colorbar(label='s4max')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Mean s4max in Latitude and Longitude Range')
plt.grid(True)
plt.show()

ValueError: Dimensions ('altitude',) not found in data dimensions ('nfile',)